In [21]:
from datetime import datetime

import numpy as np
import tensorflow as tf
import keras
from time import time

# Load the Shakespeare text file / Setup Tokenizer

In [22]:
path_to_file = keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])

/tmp/.keras/datasets/shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [23]:
# Erstellt ein `Set` mit allen Zeichen, die im Text vorkommen
set(shakespeare_text)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [24]:
# Ausgabe der Anzahl der Zeichen
len(set(shakespeare_text.lower().strip()))

39

In [25]:
# Erstellt ein `Liste` mit allen Zeichen, die im Text vorkommen
vocab = list(set(shakespeare_text.lower().strip()))

In [26]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
config = tokenizer.get_config()
tokenizer.texts_to_sequences(["hello", "world"])

[[7, 2, 12, 12, 4], [17, 4, 9, 12, 13]]

In [27]:
n_tokens = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [28]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))
print(f"{encoded[:10]}")

[20  6  9  8  3  1 19  6  3  6]


# Create Dataset

In [29]:
train_size = dataset_size * 90 // 100
print(f"train_size = {train_size:,}")

train_size = 1,003,854


In [30]:
dataset = tf.data.Dataset.from_tensor_slices(encoded)
#dataset = dataset.repeat()
n_steps = 50
dataset = dataset.window(n_steps + 1, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window_ds: window_ds.batch(n_steps + 1))

In [31]:
for x in dataset.take(2):
    print(x)

tf.Tensor(
[20  6  9  8  3  1 19  6  3  6 36  2 10 24 11 22  2 20  4  9  2  1 17  2
  1 23  9  4 19  2  2 13  1  5 10 16  1 20 14  9  3  7  2  9 18  1  7  2
  5  9  1], shape=(51,), dtype=int64)
tf.Tensor(
[ 6  9  8  3  1 19  6  3  6 36  2 10 24 11 22  2 20  4  9  2  1 17  2  1
 23  9  4 19  2  2 13  1  5 10 16  1 20 14  9  3  7  2  9 18  1  7  2  5
  9  1 15], shape=(51,), dtype=int64)


In [32]:
dataset = dataset.batch(128)

In [33]:
for x in dataset.take(1):
    print(x.shape)

(128, 51)


In [34]:
dataset = dataset.map(lambda window: (window[:, :-1], window[:, 1:]))

In [35]:
dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)

In [36]:
for x, y in dataset.take(1):
    print(f"x = {x.shape}")
    print(f"y = {y.shape}")

x = (128, 50)
y = (128, 50)


2025-04-11 19:09:52.293466: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [37]:
transform = lambda x, y: (tf.one_hot(x, depth=n_tokens), y)
dataset = dataset.map(transform)
for x, y in dataset.take(1):
    print(f"x = {x.shape}")
    print(f"y = {y.shape}")

x = (128, 50, 39)
y = (128, 50)


2025-04-11 19:09:52.448847: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-04-11 19:09:52.494346: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Create Models

In [38]:
m1 = keras.Sequential(
    [
        keras.layers.Input(shape=(None, n_tokens)),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ],
)
m1.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m1.summary()
model = m1

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,927 (273.15 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
text = "to be "
tokens = tokenizer.texts_to_sequences([text])
inp_tf = tf.constant(tokens)
inp_tf = tf.one_hot(inp_tf, depth=n_tokens)
print(inp_tf.shape)
y = model.predict(inp_tf)
print(y.shape)
y = tf.argmax(y[0,-1])

(1, 6, 39)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
(1, 6, 39)


In [43]:
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
history = m1.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
m1.save("models/shakespeare_gru_1.keras")

Epoch 1/10
7842/7842 - 51s - 6ms/step - loss: nan
Epoch 2/10
7842/7842 - 5s - 688us/step - loss: nan
Epoch 3/10


2025-04-11 19:17:38.296366: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/sequential_1_1/gru_1_1/Shape/_4]]
2025-04-11 19:17:38.296427: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 8301092065443555371
2025-04-11 19:17:38.296450: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 6212536297720087069
2025-04-11 19:17:38.296472: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14909740500617182652


KeyboardInterrupt: 

In [44]:
m2 = keras.Sequential(
    [
        keras.layers.Input(shape=(None, n_tokens)),
        keras.layers.LSTM(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ]
)
m2.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m2.summary()
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
# start_time = time.time.now()
h2 = m2.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
# runtime = time.perf_counter() - start_time
# print(f"Time {runtime:,.0f} seconds.")
m2.save("models/shakespeare_lstm_1.keras")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 128)      │        86,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,047 (355.65 KB)

 Trainable params: 91,047 (355.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
7842/7842 - 45s - 6ms/step - loss: nan
Epoch 2/10
7842/7842 - 4s - 503us/step - loss: nan
Epoch 3/10


2025-04-11 19:18:47.187476: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 3581125356803605815
2025-04-11 19:18:47.187533: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 12625261226567069429
2025-04-11 19:18:47.187560: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 2616091068365635478


7842/7842 - 42s - 5ms/step - loss: nan


FileNotFoundError: [Errno 2] No such file or directory: 'models/shakespeare_lstm_1.keras'

In [ ]:
m3 = keras.Sequential(
    [
        keras.layers.Input(shape=(None, n_tokens)),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ]
)
m3.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m3.summary()
model = m3

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
start_time = datetime.now()
history3 = m3.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
elapsed_time3 = datetime.now() - start_time
print(f"Training completed in {elapsed_time3} seconds.")
m3.save("models/shakespeare_gru_2.keras")

In [ ]:
import matplotlib.pyplot as plt

# Extract the loss values from the three history objects
loss1 = history.history['loss']
loss2 = h2.history['loss']
loss3 = history3.history['loss']

# Plot the training losses
plt.plot(loss1, label="Run 1 (GRU)", linestyle='-', color='blue')
plt.plot(loss2, label="Run 2 (LSTM)", linestyle='--', color='orange')
plt.plot(loss3, label="Run 3 (Stacked GRU)", linestyle='-.', color='green')

# Add labels, title, and legend
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Comparison of Training Histories")
plt.legend()

# Display the plot
plt.show()